# Exercise 03
## Lars Kühmichel, Nicolas Wolf

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import scipy.stats as scistats

In [2]:
plt.rc("figure", dpi=150)
plt.rc("legend", fontsize=6)

## Task 1: Univariate AR Models

Consider the time series `DLPFC`

As always, load and inspect the data:

In [3]:
df = pd.read_csv("data/ex3file1.csv")

In [4]:
df.head()

,DLPFC,DLPFC.1,Parietal,Parietal.1
0,2.28550,-0.29516,-3.0362,-4.51840
1,0.85457,-2.63380,-4.5223,-1.90250
2,-0.26151,-3.08110,-4.7146,0.47887
3,-1.19410,-2.62330,-3.9754,1.92820
4,-1.27900,-1.61420,-2.0104,3.77230


In [ ]:
df["DLPFC"].plot()
plt.show()

### 1. Compute the log-likelihood of an AR(4) model. Please write down explicitly.

According to the script, the log-likelihood is given as:

$$
\log L(\{ \alpha_i \}, \sigma) = - \frac{T-p}{2} \log (2 \pi) - \frac{T-p}{2} \log(\sigma^2) - \frac{1}{2} \underline{\varepsilon}^T \underline{\varepsilon} \sigma^{-2}
$$

where $\{ \alpha_i \}$ is the set of model coefficients, $\sigma$ is the standard deviation of the process noise $\underline{\varepsilon} \sim \mathcal{N}(0, \sigma^2 \underline{I})$, $T$ is the length of the time series, and $p$ is the order of the AR process.


In [ ]:
model = sm.tsa.AutoReg(endog=df["DLPFC"], lags=4, trend="n")
result = model.fit()

p = 4
T = len(df)
sigma = np.sqrt(result.sigma2)
epsilon = result.resid
ll = -0.5 * (T - p) * np.log(2 * np.pi) - 0.5 * (T - p) * np.log(sigma ** 2) - 0.5 * np.dot(epsilon, epsilon) * sigma ** (-2)
llf = result.llf

print(f"Library: {llf:.2f}")
print(f"Ours:    {ll:.2f}")
print(f"Close?   {np.isclose(llf, ll)}")

The log-likelihood is quite small. However, the model yields a good fit:

In [ ]:
df["DLPFC"].plot()
sns.lineplot(x=df.index, y=result.predict(), label="AR(4) Fit")
result.forecast(steps=30).plot(color="C1", ls=":", alpha=0.75, label="AR(4) Forecast")
plt.legend()
plt.show()

### 2. Plot the residuals of the model in a histogram. What do they look like? What do you expect?

The model still deviates from the true signal especially at the peaks. Thus, we would expect significant residuals in those areas. Residuals should look like the multivariate normal noise $\underline{\varepsilon}$, which we confirm with the histogram and an overlaid distribution kde plot. Notably, the residuals are not time-dependent.

In [ ]:
residuals = df["DLPFC"] - result.predict()
sns.lineplot(x=df.index[model.hold_back:], y=result.resid)
plt.title("Residuals")
plt.show()

In [ ]:
sns.histplot(data=residuals, stat="density")
sns.kdeplot(data=residuals, fill=True, cut=0, label="Observed", ax=plt.gca())
x = np.linspace(*plt.xlim(), num=100)
y = scistats.norm.pdf(x, scale=sigma)
plt.plot(x, y, ls="--", alpha=0.5, color="black", label="Theory")
plt.xlabel(r"$\varepsilon$")
plt.title("Distribution of Residuals")
plt.legend()
plt.show()

### 3. Compute the log-likelihood of an AR($n$) model, with $n$ ranging from $1...5$.

We compute it from $1...100$:

In [ ]:
ns = np.arange(start=1, stop=100 + 1)

results = []
lls = []

for n in ns:
    model = sm.tsa.AutoReg(endog=df["DLPFC"], lags=n, trend="n")
    result = model.fit()
    results.append(result)
    lls.append(result.llf)

likelihoods = pd.DataFrame({"log-likelihood": lls}, index=ns)

likelihoods.plot()
plt.show()

### How does the likelihood change when you increase the order of the model?

The log-likelihood increases as the number of parameters increases. This means the model will more accurately capture the training set, which is logical since the model gains explanatory power. However, a large number of parameters will lead to overfitting.

### 4. Bonus Exercise: Increasing the capacity of the model is likely to increase its explanatory power, but it is important to explore the tradeoff between this and the increase in model parameters. Determine the optimal order $p$ of the AR model by computing the log-likelihood ratio test statistic.

The p-value for the likelihood-ratio-test gives the probability that the hypothesis "The unrestricted model improves on the restricted one" is false. The optimal order is thus that of the restricted model at the first (desired) maximum of p.

In [ ]:
from statsmodels.base.model import LikelihoodModelResults as LMR


def lr_test(restricted: LMR, unrestricted: LMR) -> float:
    """ Perform a Likelihood-Ratio-Test between a restricted and unrestricted model """
    # determine the appropriate degrees of freedom
    dof = restricted.df_resid - unrestricted.df_resid
    # likelihood-ratio-test-statistic or "D" on the sheet:
    stat_lr = -2 * (restricted.llf - unrestricted.llf)
    # perform chi-squared test (compute p-value)
    return scistats.chi2.sf(stat_lr, df=dof)

The optimal order is 2. An argument could also be made for 20, 30, 63, 71 and 98 since these have even higher p-values, depending on the statistical certainty requirement. Orders 28, 29 and 30 also stand out because they successively yield good p-values.

In [ ]:
ps = []

for restricted, full in zip(results[:-1], results[1:]):
    ps.append(lr_test(restricted, full))

peaks = []
m = ps[0]
for i, p in enumerate(ps):
    if p > m:
        m = p
        peaks.append(i + 1)


sns.lineplot(x=ns[:-1], y=ps, label="p-Values")
print(f"Marked Peaks are at: {peaks}")
for p in peaks:
    plt.axvline(p, ls=":", lw=1, alpha=0.5)

plt.xlabel("Order of Restricted Model")
plt.ylabel("p")
plt.yscale("log")
plt.legend(loc="lower right")
plt.title("Likelihood-Ratio-Test Results")
plt.show()

## Task 2: Multivariate (vector) AR (=VAR) processes

Now use all four time series `DLPFC1, DLPFC2, Parietal1, Parietal2`.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4), sharex="all", sharey="all")
df[["DLPFC", "DLPFC.1"]].plot(ax=axes[0])
df[["Parietal", "Parietal.1"]].plot(ax=axes[1])
plt.show()


### 1. Estimate a VAR(1) model by performing multivariate regression on the 4-variate time series.

In [ ]:
model = sm.tsa.VAR(df)
result = model.fit()

print(result.summary())

### What do the coefficients in matrix $A$ tell you about the coupling between the DLPFC and parietal cortex?

The `DLPFC` and `Parietal` series are weakly coupled, since the coupling coefficients are `0.016` and `-0.012` respectively. We can also reason that the absolute influence `DLPFC` has on `Parietal` is around the same strength as the other way around, i.e. neither is particularly causative of the other. The strongest (non-self) coupling exists from `Parietal.1` to `DLPFC`, with a coefficient of `0.154`.

In [ ]:
names = ["DLPFC", "DLPFC.1", "Parietal", "Parietal.1"]
A = pd.DataFrame(data=np.squeeze(result.coefs), index=names, columns=names)
A

### Is the resulting VAR(1) model stationary or not?

We would expect the model to be stationary, since the underlying time series were stationary. Since the model fits well, this is a reasonable assumption.

In [ ]:
y = np.array(df.iloc[-1])[None, :]
in_sample = model.predict(result.params)
in_sample = pd.DataFrame(data=in_sample, index=df.index[1:], columns=df.columns)

fig, axes = plt.subplots(nrows=1, ncols=4, sharex="all", sharey="all", figsize=(16, 4))

for i, col in enumerate(df.columns):
    ax = axes[i]
    df[col].plot(ax=ax, label="True")
    in_sample[col].plot(ax=ax, label="Predicted")
    ax.set_title(col)
    ax.legend()

fig.text(0.5, 0.01, "Time", ha="center", va="top")
fig.text(0.01, 0.5, "Time Series", rotation="vertical", va="center", ha="right")
fig.suptitle("VAR(1) Fits")
plt.tight_layout()
plt.show()

## Task 3: AR Poisson Processes

### 1. Create your own second order Poisson time series with the given parameters.

According to equation 7.36 in the script:
$$
\log \underline{\mu}_t = \underline{a}_0 + \sum_{m=1}^{M} \underline{A}_m \underline{c}_{t-m}
$$

With $c_{it} \sim \operatorname{Poisson}(\mu_{it}) \quad \forall i$.

In [ ]:
def create_AR_poisson_data(T, mu0, As, a0):
    mu = mu0
    c_arr = np.zeros(shape=(T, len(a0)))
    c_arr[0] = np.random.poisson(mu, 2)
    
    M = len(As)
    for t in range(1, T):
        log_mu_t = a0.copy()
        for m in range(1, min(M, t)+1):
            log_mu_t += As[m-1].dot(c_arr[t-m])
        mu = np.exp(log_mu_t)
        c_arr[t] = np.random.poisson(mu, 2)
        
    return c_arr

In [ ]:
T = 1000
A1 = np.array([
    [0.2, -0.1],
    [0.1, 0.1]
])
A2 = np.array([
    [0.1, -0.1],
    [0.1, 0.1]
])
mu0 = np.array([0.5, 0.5]).T
a0 = np.array([0.0, 0.0]).T
ts = create_AR_poisson_data(T, mu0, [A1, A2], a0)

In [ ]:
plt.plot(ts[:,0], label=r"$c_1$")
plt.plot(ts[:,1], label=r"$c_2$")
plt.xlabel("t")
plt.ylabel("c")
plt.legend()
plt.title("2nd order AR-Poisson time series")
plt.show()

In [ ]:
sns.histplot(data=ts, stat="density")
sns.kdeplot(data=ts, fill=True, bw_adjust=3, cut=0, ax=plt.gca())
plt.legend(labels=[r"$c_1$", r"$c_2$"])
plt.show()

### 2. Given the data generated in (1), vary the parameters $A_1(1, 1)$ and $A_2(2, 1)$ between 0 and 0.4 with 0.01 increments. For each parameter value pair, compute the log-likelihood of the data (keeping all other parameters fixed).

The log likelihood for a poisson process is given by equation (7.37) in the script:
$$
\log p(\{\underline{c}_t\} | \underline{A}) = \log\left[\prod_{t=M+1}^{T}\prod_{i=1}^{p}\frac{\mu_{it}^{c_{it}}}{c_{it}!}e^{-\mu_{it}}\right] = \sum_{t=M+1}^{T}\sum_{i=1}^{p} c_{it}\log\mu_{it}-\mu_{it}
$$

In [ ]:
def log_likelihood_AR_Poisson(data, As, a0):
    T = len(data)
    M = len(As)
    ll = 0
    mu_arr = np.zeros(shape=data.shape)
    for t in range(M, T):
        log_mu_t = a0.copy()
        for m in range(1, min(t, M)+1):
            log_mu_t += As[m-1].dot(data[t-m])
        
        ll += np.sum(data[t]*log_mu_t-np.exp(log_mu_t))

        
    return ll

In [ ]:
parameter_range = np.linspace(0, 0.4, int(0.4/0.01)+1)
lls = np.zeros(shape=(len(parameter_range), len(parameter_range)))
for i, a1 in enumerate(parameter_range):
    for j, a2 in enumerate(parameter_range):
        A1 = np.array([
            [a1, -0.1],
            [0.1, 0.1]
        ])
        A2 = np.array([
            [0.1, -0.1],
            [a2, 0.1]
        ])
        lls[i,j] = log_likelihood_AR_Poisson(ts, [A1, A2], a0)

In [ ]:
argmax_indices = np.unravel_index(lls.argmax(), lls.shape)
print(argmax_indices, (parameter_range[argmax_indices[0]], parameter_range[argmax_indices[1]]), lls[argmax_indices])
print((20,10), (parameter_range[20], parameter_range[10]), lls[20,10])

### Plot the log-likelihood landscape surface as a function of these two parameters.

In [ ]:
ax = plt.gca()
plt.imshow(lls.T, cmap='hot', interpolation='nearest', extent=[0,0.4,0,.4], origin='lower')
plt.colorbar()
plt.scatter(parameter_range[argmax_indices[0]], parameter_range[argmax_indices[1]], label="argmax")
plt.scatter(0.2, 0.1, label="real parameter")
plt.xlabel(r"$A_1(1,1)$")
plt.ylabel(r"$A_2(2,1)$")
plt.legend()
plt.title("Log-likelihood landscape")
plt.show()

### Does the real parameter pair value correspond (or is close) to an extreme point in the approximate log-likelihood landscape? What kind of extreme point is it?

The extreme point of L($A_1(1,1)$, $A_2(2,1)$) is at $(0.18, 0.11)$ which is very close to the real parameters $(0.2, 0.1)$. Since this extreme point is a maximum, the argmax parameters are the parameters for which the real data are most probable.

### Does the real parameter pair value correspond (or is close) to an extreme point in the approximate log-likelihood landscape? What kind of extreme point is it?

The extreme point of L($A_1(1,1)$, $A_2(2,1)$) is at $(0.18, 0.11)$ which is very close to the real parameters $(0.2, 0.1)$. Since this extreme point is a maximum, the argmax parameters are the parameters for which the real data are most probable.